In [2]:
# MatchJointerRaw test
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/matchjointer1-test.yaml')
model = loadModel(config['model'])


c_time, c_pitch, c_velocity = torch.rand(1, 10) * 10000, torch.randint(128, (1, 10)), torch.rand(1, 10) * 127
s_time, s_pitch, s_velocity = torch.rand(1, 6) * 10000, torch.randint(128, (1, 6)), torch.rand(1, 6) * 127

matching, c_vec, s_vec = model(c_time, c_pitch, c_velocity, s_time, s_pitch, s_velocity)

print('matching:', matching.shape)
print('c_vec:', c_vec.shape)
print('s_vec:', s_vec.shape)


matching: torch.Size([1, 6, 10])
c_vec: torch.Size([1, 6, 128])
s_vec: torch.Size([1, 128, 10])


In [5]:
# dataset
import os
from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset


DATA_DIR = os.environ.get('DATA_DIR')

config = Configuration.create('configs/matchjointer1-test.yaml')
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')

print('n_examples:', train.dataset.n_examples, val.dataset.n_examples)

it = iter(train)
batch = next(it)
print('tensors:', [(tensor.shape, tensor.dtype) for tensor in batch['criterion']], [(tensor.shape, tensor.dtype) for tensor in batch['sample']], batch['ci'].shape)

next(it)
next(it)
next(it)
batch = next(it)
print('cis:', batch['ci'][:, -15:])
print('sample_mask:', batch['sample'][1] > 0)
print('pitch-c', batch['criterion'][1][:, :20])
print('pitch-s', batch['sample'][1][:, -20:])
print('time-s:', batch['sample'][0][:, -20:])


n_examples: 14997 3809
tensors: [(torch.Size([4, 64]), torch.float32), (torch.Size([4, 64]), torch.int64), (torch.Size([4, 64]), torch.float32)] [(torch.Size([4, 64]), torch.float32), (torch.Size([4, 64]), torch.int64), (torch.Size([4, 64]), torch.float32)] torch.Size([4, 64])
cis: tensor([[ 3,  4,  5,  6,  7,  8, 10,  9, 11, 11, 13, 12, 14, 15, 16],
        [ 4,  5,  6,  7,  8, 10,  9, 11, 11, 13, 12, 14, 15, 16, 17],
        [ 5,  6,  7,  8, 10,  9, 11, 11, 13, 12, 14, 15, 16, 17, 19],
        [ 6,  7,  8, 10,  9, 11, 11, 13, 12, 14, 15, 16, 17, 19, 18]])
sample_mask: tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False,  True,  True,  True,
          True,  True,  True

In [2]:
# MatchJointer1Loss
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/matchjointer1-test.yaml')
model = loadModel(config['model'], postfix='Loss')

loss, metric = model(batch)

print('loss:', loss)
print('metric:', metric)


loss: tensor(0.9049, grad_fn=<AddBackward0>)
metric: {'acc_full': 0.08108108108108109, 'acc_c1': 0.08108108108108109, 'acc_tail8': 0.03125, 'acc_tip': 0.0}


In [2]:
# tar * tar
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/matchjointer3-test.yaml')
model = loadModel(config['model'])

criterion, sample = batch['criterion'], batch['sample']
matching, code_src, code_tar = model(*criterion, *sample)

code_tar_trans = code_tar.transpose(-2, -1)
prod = code_tar.matmul(code_tar_trans)

print('prod:', prod)


prod: tensor([[[1.0000, 0.4778, 0.6071,  ..., 0.3946, 0.4345, 0.4036],
         [0.4778, 1.0000, 0.2963,  ..., 0.3182, 0.3821, 0.3645],
         [0.6071, 0.2963, 1.0000,  ..., 0.1527, 0.2537, 0.1823],
         ...,
         [0.3946, 0.3182, 0.1527,  ..., 1.0000, 0.8633, 0.8685],
         [0.4345, 0.3821, 0.2537,  ..., 0.8633, 1.0000, 0.8511],
         [0.4036, 0.3645, 0.1823,  ..., 0.8685, 0.8511, 1.0000]],

        [[1.0000, 0.4223, 0.5442,  ..., 0.2787, 0.2422, 0.2816],
         [0.4223, 1.0000, 0.4499,  ..., 0.2012, 0.1890, 0.2677],
         [0.5442, 0.4499, 1.0000,  ..., 0.3435, 0.3205, 0.3489],
         ...,
         [0.2787, 0.2012, 0.3435,  ..., 1.0000, 0.9242, 0.8962],
         [0.2422, 0.1890, 0.3205,  ..., 0.9242, 1.0000, 0.8573],
         [0.2816, 0.2677, 0.3489,  ..., 0.8962, 0.8573, 1.0000]],

        [[1.0000, 0.3276, 0.5554,  ..., 0.2657, 0.2640, 0.2050],
         [0.3276, 1.0000, 0.3875,  ..., 0.2441, 0.2402, 0.1193],
         [0.5554, 0.3875, 1.0000,  ..., 0.3520, 0.34